# Homework 01

**1**. (25 points)

The code below gives five "documents" with titles in `titles` and text in `contents`. 

- Convert each text into "words" by converting to lower case, removing punctuation and splitting on whitespace
- Make a list of all unique "words" in any of the texts
- Create an pandas DataFrame whose rows are words, columns are titles, and values are counts of the word in the document
- Add a column `total` that counts the total number of occurrences for each word across all documents
- Show the rows for the 5 most commonly used words

In [1]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups(subset='train')
target_names = twenty['target_names']
titles = [target_names[i] for i in twenty['target'][2:7]]
contents = twenty['data'][2:7]

In [2]:
import string
import numpy as np
import pandas as pd
def cleanWords(txt):
    cleanedTxt=txt.strip().lower() \
    .translate(str.maketrans('','', string.punctuation)) \
    .replace('\n','').split()
    return cleanedTxt

In [3]:
doc_words = [cleanWords(content) for content in contents]
vocab = set([word for words in doc_words for word in words])

In [4]:
table = np.zeros((len(vocab), len(contents)), dtype='int')
for i, word in enumerate(vocab):
    for j, doc in enumerate(doc_words):
        table[i, j] = doc.count(word)

In [5]:
df_wordcount = pd.DataFrame(table, columns='doc1 doc2 doc3 doc4 doc5'.split(), index=vocab)
df_wordcount['total'] = df_wordcount.sum(numeric_only=True ,axis=1)
df_wordcount.sort_values('total', ascending=False).head(5)

,doc1,doc2,doc3,doc4,doc5,total
the,17,3,7,16,0,43
of,5,1,1,16,3,26
to,5,1,1,9,2,18
a,8,1,3,2,2,16
i,7,0,1,4,3,15


**2**. (75 points)

A Caesar cipher is a very simple method of encoding and decoding data. The cipher simply replaces characters with the character offset by $k$ places. For example, if the offset is 3, we replace `a` with `d`, `b` with `e` etc. The cipher wraps around so we replace `y` with `b`, `z` with `c` and so on. Punctuation, spaces and numbers are left unchanged.

- Write a function `encode` that takes as arguments a string and an integer offset and returns the encoded cipher.
- Write a function `decode` that takes as arguments a cipher and an integer offset and returns the decoded string. 
- Write a function `auto_decode` that takes as argument a cipher and uses a statistical method to guess the optimal offset to decode the cipher, assuming the original string is in English which has the following letter frequency:

```python
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}
```

- Encode the following nursery rhyme using a random offset from 10 to 20, then recover the original using `auto_decode`:

```text
Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane.
```

In [6]:
rhythm="""Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane."""

In [7]:
def translate(letter, offset: int):
    if letter.isupper():
        converted=letter.lower()
        converted_num=((ord(converted)-97)+offset)%26+97
        newLetter = chr(converted_num).upper()
    else:
        converted_num=((ord(letter)-97)+offset)%26+97
        newLetter = chr(converted_num)
    return newLetter

In [8]:
def encode(string, offset):
    return ''.join(translate(i, offset) if i.isalpha() else i for i in string)

In [9]:
def decode(string, offset):
    return ''.join(translate(i, -offset) if i.isalpha() else i for i in string)

In [10]:
np.random.seed(12)
offset = np.random.randint(10,20)
offset

16

In [11]:
cipher = encode(rhythm, offset)
cipher

'Rqq, rqq, rbqsa ixuuf,\nXqlu oek qdo meeb?\nOui, iyh, oui, iyh,\nJxhuu rqwi vkbb;\nEdu veh jxu cqijuh,\nQdt edu veh jxu tqcu,\nQdt edu veh jxu byjjbu reo\nMxe bylui temd jxu bqdu.'

In [12]:
decode(cipher, offset)

'Baa, baa, black sheep,\nHave you any wool?\nYes, sir, yes, sir,\nThree bags full;\nOne for the master,\nAnd one for the dame,\nAnd one for the little boy\nWho lives down the lane.'

Find most likely offset to auto decode based on probability

In [13]:
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}

In [21]:
from collections import Counter
def auto_decode(cipher):
    #remove punc and lowercase all in the cipher
    cleaned_cipher = cipher.lower() \
        .translate(str.maketrans('','', string.punctuation)) \
        .replace('\n','').replace(' ','')   
    #find expected frequency given the length of the cipher
    freq_english= {k:v*len(cleaned_cipher) for k,v in freq.items()}
    #assume different offset
    chi_sq_lst = []
    for j in range(1,26):
        guess=decode(cleaned_cipher, j)
        freq_cipher= dict(Counter(guess))

        #find min chi squared
        chi_sq_sum=0
        for i in freq_cipher.keys():
            chi_sq = (freq_cipher[i] - freq_english[i])**2/freq_english[i]
            chi_sq_sum += chi_sq
        chi_sq_lst.append(chi_sq_sum)
    return decode(cipher, np.argmin(chi_sq_lst)+1)

In [22]:
auto_decode(cipher)

'Baa, baa, black sheep,\nHave you any wool?\nYes, sir, yes, sir,\nThree bags full;\nOne for the master,\nAnd one for the dame,\nAnd one for the little boy\nWho lives down the lane.'